<a href="https://colab.research.google.com/github/pappajohn/fantasy-sports/blob/main/TradeWinnerEvaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Let's see how trades are going!

In [ ]:
# Install libraries
%pip install espn_api

import pandas as pd
import requests
import json

from espn_api.football import League
from collections import defaultdict

In [ ]:
# Authenticate in to the private league at https://github.com/pappajohn/fantasy-sports

league_id = 486993
espn_s2 = 'AEC1VWgqxbVS...' #REPLACE
swid = '{0CC7C368-D8D1-4ED...' #REPLACE
year = 2025 # Replace with the desired latest year
league = League(league_id=league_id, espn_s2=espn_s2, swid=swid, year=year)

kw = 4567048
justinj = 4262921
jamo = 4426388

In [ ]:
# function to get score

def points_since_trade(player_id, first_week_of_counting):
  d = league.player_info(playerId=player_id).stats
  total_points = sum(v['points'] for k, v in d.items() if k > first_week_of_counting and 'points' in v)
  #name = player(playerId=player_id).player_name NO SUCH OBJECT EXISTS
  return total_points


In [ ]:
# Get cumulative score for each traded player

print("JJ total = " + str(points_since_trade(justinj, 3)))
print("Jamo total = " + str(points_since_trade(jamo, 3)))
print("KW3 total = " + str(points_since_trade(kw, 3)))

JJ total = 17.6
Jamo total = 4.7
KW3 total = 11.5


In [ ]:
# Compare to score of highest projected waiver player at position (need to figure out the interweek part)

def waiver_alternates():
  for pos in ['QB', 'RB', 'WR', 'TE']:
    players = league.free_agents(position=pos)
    #print(players)
    sorted_players = sorted(players, key=lambda player: player.projected_points, reverse=True)
    #print(sorted_players)
    print("The best WW " + str(pos) + " was " + str(sorted_players[0].name) + " with " + str(sorted_players[0].points) + " and " + str(sorted_players[0].projected_points) + " projected")


In [ ]:
waiver_alternates()

The best WW QB was Geno Smith with 9.78 and 17.51 projected
The best WW RB was Devin Singletary with 2.8 and 6.33 projected
The best WW WR was Dontayvion Wicks with 2.9 and 7.99 projected
The best WW TE was Dalton Schultz with 5.5 and 7.22 projected


#Unfinished work
Compare performance of acquired player to best alternative (highest projected scoring player at similar position on owner bench or waivers)

Do multi-week sums of best alternative player

In [ ]:
## Reverse ID lookup from here https://github.com/mattabets/espn_api_player_id_reverse_lookup/blob/main/README.md

def espn_api_player_id_reverse_lookup(player_name):
    TEAM_API = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams"
    response = requests.get(TEAM_API)

    if response.status_code == 200:
        teams_data = response.json()
        teams = teams_data["sports"][0]["leagues"][0]["teams"]

        roster = []

        # Iterate over each team
        for team in teams:
            team_id = team["team"]["id"]
            team_name = team["team"]["abbreviation"]

            # Fetch roster data for the team
            roster_url = f"{TEAM_API}/{team_id}?enable=roster"
            roster_response = requests.get(roster_url)

            if roster_response.status_code == 200:
                roster_data = roster_response.json()

                # Extract athlete information
                athletes = [
                    {
                        "id": athlete["id"],
                        "team": team_name,
                        "fullName": athlete["fullName"],
                    }
                    for athlete in roster_data["team"]["athletes"]
                ]

                # Add athlete information to the roster list
                roster.extend(athletes)

        # Search for player in the roster
        found_players = [
            {"id": player["id"], "team": player["team"]}
            for player in roster
            if player["fullName"].lower() == player_name.lower()
        ]

        if found_players:
            # Return the player ID instead of printing it
            return found_players[0]['id']  # Assuming only one player is found
        else:
            print("Player not found.")
            return None
    else:
        print("Failed to fetch NFL teams data.")
        return None




In [ ]:
# Call the function with the desired player name
player_name = 'Patrick Mahomes'
player_id = espn_api_player_id_reverse_lookup(player_name)

# Print the player ID if found
if player_id:
    print("Player ID:", player_id)
else:
    print("Player not found.")

Player ID: 3139477


In [ ]:
# Find the best free agent at position X in a fixed week

## Need to make sure we have the right week

## Then generate a list and grab the points for the highest projected player

## Here's a one off version, we need to do it programmatically

players = league.free_agents(position='QB')

# Sort players by projected points in descending order
sorted_players = sorted(players, key=lambda player: player.projected_points, reverse=True)

print("Free Agents (QB) sorted by Projected Points:")
for player in sorted_players:
    print(f"Player({player.name}, points:{player.points}, projected:{player.projected_points})")

print(sorted_players[0].points)

Free Agents (QB) sorted by Projected Points:
Player(Geno Smith, points:9.78, projected:17.51)
Player(Kyler Murray, points:16.1, projected:16.31)
Player(Matthew Stafford, points:27.4, projected:16.0)
Player(Tua Tagovailoa, points:0, projected:15.63)
Player(Bryce Young, points:10.3, projected:15.34)
Player(Michael Penix Jr., points:18.72, projected:14.55)
Player(Carson Wentz, points:21.2, projected:14.36)
Player(Jake Browning, points:0, projected:14.1)
Player(Dak Prescott, points:30.96, projected:14.08)
Player(Jaxson Dart, points:19.84, projected:14.02)
Player(Spencer Rattler, points:13.94, projected:13.78)
Player(Sam Darnold, points:16.08, projected:13.02)
Player(Aaron Rodgers, points:12.8, projected:12.69)
Player(Cam Ward, points:4.52, projected:12.4)
Player(Joe Flacco, points:1.26, projected:11.63)
Player(Jalen Milroe, points:0.0, projected:0.16)
Player(J.J. McCarthy, points:0.0, projected:0.0)
Player(Marcus Mariota, points:16.24, projected:0.0)
Player(Russell Wilson, points:0.32, pro

In [ ]:
waiver_alternate("QB")

Player(Kyler Murray, points:16.1, projected:16.31)


##Resources to learn more

In [ ]:
# If you want to explore what the league object looks like, do this
for attribute_name in dir(league):
  try:
    attribute_value = getattr(league, attribute_name)
    print(f"{attribute_name}: {attribute_value}")
    print(" ")
  except Exception as e:
    print(f"Could not access attribute {attribute_name}: {e}")
    print(" ")